In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import json
import collections
import re
from tqdm import tqdm
import itertools

from itertools import islice
from collections import Counter, defaultdict

from itertools import chain
import ast


import statistics
import numpy as np

from numpy import core
import novelty_ipc
from novelty_ipc import Newness, Surprise, Uniqueness, Difference



In [2]:
import importlib
importlib.reload(novelty_ipc)

<module 'novelty_ipc' from 'c:\\Users\\edgar\\OneDrive\\Bureau\\Ecole\\HEC\\A24\\BrevetNLP\\PatentNovelty\\ipcMetrics\\novelty_ipc.py'>

In [4]:
# Importation données
# toEval = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/toEval/2016_G06F_patents_toEval.csv') # /home/edgarlanoue/data/csv/toEval/2016_G06F_patents_toEval.csv 'C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/toEval/2016_G06F_patents_toEval.csv'
# KS = pd.read_csv('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/KS/2016_1115_G06F_KS_raw.csv') # /home/edgarlanoue/data/csv/KS/2016_1115_G06F_KS_raw.csv C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/KS/2016_1115_G06F_KS_raw.csv

# toEval.to_pickle("C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_G06F_patents_toEval.pkl")
toEval = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_G06F_patents_toEval.pkl')
KS = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_1115_G06F_KS_raw.pkl')

print("toEval lenght: ", len(toEval))
print("KS lenght: ", len(KS))



# Création différents jdD
toEval_ipc = list([ast.literal_eval(i) for i in toEval.sec_ipc])
KS_ipc = list(set(chain.from_iterable([ast.literal_eval(s) for s in KS.sec_ipc])))
KS_sec_ipc = list(pd.Series([ast.literal_eval(i) for i in KS.sec_ipc]))



toEval lenght:  9234
KS lenght:  173662


In [5]:
# toEval = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_G06F_patents_toEval.pkl') 
ES = pd.read_pickle('C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/pickledDF/2016_1115_G06F_ES_raw.pkl')
print("toEval lenght: ", len(toEval))
print("KS lenght: ", len(ES))

# Création différents jdD
# toEval_ipc = list([ast.literal_eval(i) for i in toEval.sec_ipc])
ES_ipc = list(set(chain.from_iterable([ast.literal_eval(s) for s in ES.sec_ipc])))
ES_sec_ipc = list(pd.Series([ast.literal_eval(i) for i in ES.sec_ipc]))

toEval lenght:  9234
KS lenght:  431267


### Newness

In [196]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Newness

instanceNewness = Newness(list_unique_known=KS_ipc, list_new=toEval_ipc)

In [197]:
scoreNewness = instanceNewness.novelty_newness(thr=0)

100%|██████████| 9234/9234 [00:01<00:00, 5752.73it/s]


In [198]:
print(scoreNewness[0][10:16])
print(scoreNewness[1][10:16])
print([1 if (scoreNewness[1][i] == 1 and scoreNewness[2][i] == 1) else 0 for i in range(len(scoreNewness[0]))][:6])

[0.0, 0.0, 0.16666666666666666, 0.0, 0.0, 0.0]
[0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0]


In [200]:
print(Counter('>0' if score > 0 else '0' for score in scoreNewness[0]))
print(Counter(scoreNewness[1]))
print(Counter([1 if (scoreNewness[1][i] == 1 and scoreNewness[2][i] == 1) else 0 for i in range(len(scoreNewness[0]))]))
np.mean(scoreNewness[0])


Counter({'0': 8843, '>0': 391})
Counter({0: 8843, 1: 391})
Counter({0: 9226, 1: 8})


0.014945940307376301

In [205]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Uniqueness
from novelty_ipc import Difference

### Uniqueness

In [206]:
from novelty_ipc import Uniqueness

instanceUnique = Uniqueness(list_new=toEval_ipc, list_known=KS_sec_ipc)

In [207]:
scoreUnique = instanceUnique.dist_toProto(thr=0.5)

100%|██████████| 9234/9234 [00:00<00:00, 841832.83it/s]


In [208]:
print(scoreUnique[0][10:15])
print(scoreUnique[1][10:15])

[1.0, 1.0, 1.0, 0.0, 0.0]
[1, 1, 1, 0, 0]


In [209]:
Counter(scoreUnique[1])

Counter({1: 8184, 0: 1050})

In [210]:
np.mean(scoreUnique[0])

0.8949833497429338

### Difference

In [212]:
################################################
#do not forget to take out random state!
#from PatentNovelty.ipcMetrics.novelty_ipc
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Uniqueness
from novelty_ipc import Difference

instanceDiff10 = Difference(list_new=toEval_ipc, list_known=KS_sec_ipc, N=10, nb_K=1000, chunksize=500)
# instanceDiff100 = Difference(list_new_Q=KS_sec_ipc, list_known_P=KS_sec_ipc, N=100, nbKS=1000)

In [213]:
print("Seuil avec 10: ", instanceDiff10.neighbor_dist)
# print("Seuil avec 100: ", instanceDiff100.neighbor_dist)

Seuil avec 10:  0.05764059523809524


In [131]:
rTA10 = instanceDiff10.ratio_toAll(n=50)

100%|██████████| 50/50 [00:09<00:00,  5.07it/s]


In [139]:
rTA10_matrix = instanceDiff10.ratio_toAll_matrix(n=50) # Plus rapide (de moitié), ajout de chunksize pour éviter problème de RAM!

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:08<00:00,  8.25s/it]


In [140]:
print(np.mean(rTA10[0]))
print(np.mean(rTA10_matrix[0]))

0.9837423270490953
0.9837423270490953


In [141]:
print(np.mean(rTA10[1]))
print(np.mean(rTA10_matrix[1]))

0.9
0.9


In [ ]:
rTA10_matrix_n1000 = instanceDiff10.ratio_toAll_matrix(n=1000)
rTA10_matrix_nall = instanceDiff10.ratio_toAll_matrix(n="all")

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 19/19 [08:08<00:00, 25.69s/it]


In [ ]:
rTA10_test_nall = instanceDiff10.ratio_toAll(n="all") # prend 30 minutes+, version matrix 3 fois moins longue.

In [152]:
print(np.mean(rTA10_matrix[0]))
print(np.mean(rTA10_matrix_n1000[0]))
print(np.mean(rTA10_matrix_nall[0]))

0.9837423270490953
0.987445186626896
0.9887992441791914


In [193]:
rTN = instanceDiff10.ratio_toNeighbors(n=500)

100%|██████████| 500/500 [01:33<00:00,  5.35it/s]


In [214]:
rTN_matrix = instanceDiff10.ratio_toNeighbors_matrix(n=500)

100%|██████████| 1/1 [00:19<00:00, 19.66s/it]


In [215]:
# print(np.mean(rTN[0]))
print(np.mean(rTN_matrix[0]))

0.44339999999999996


In [196]:
print(np.mean(rTN[1]))
print(np.mean(rTN_matrix[1]))

0.342
0.342


# Surprise

In [176]:
import novelty_ipc
import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Surprise

instanceSurp = Surprise(list_expec=ES_sec_ipc, list_new=toEval_ipc)

In [177]:
scoreSurpNew = instanceSurp.surprise_new()

100%|██████████| 9234/9234 [00:00<00:00, 318898.34it/s]


In [216]:
print(Counter(scoreSurpNew[1]))
print(np.mean(scoreSurpNew[0]))
print(np.mean(list(instanceSurp.dict_expec.values())))

Counter({0: 6504, 1: 2730})
0.1820793951073353
5.0751831145250845


In [179]:
scoreSurpDiv = instanceSurp.surprise_div(0)

100%|██████████| 9234/9234 [00:00<00:00, 154316.51it/s]


In [180]:
print(np.mean(scoreSurpDiv[0]))
print(np.mean(scoreSurpDiv[1]))

4.11553224368677
0.6846437080355209


# Train 
Fixing thresholds as mean + standard deviation
Taking 2 IPC classes: B32B, C07D

In [26]:
pathData = "/home/edgarlanoue/data/csv" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/trainData"
pathOutput = "/home/edgarlanoue/data/csv/ipcMetrics/train" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/trainData/ipcMetrics"
listIPC =  ["C07D", "B32B"]
listYear = range(2011, 2017)

In [35]:
# # import novelty_ipc
# import importlib
importlib.reload(novelty_ipc)
from novelty_ipc import Newness, Uniqueness, Difference, Surprise, compute_scores, ipcMetrics

In [36]:
ipcMetrics(listIPC, listYear, pathData=pathData, pathOutput=pathOutput)

C07D
     2011




100%|██████████| 1121/1121 [00:00<00:00, 10913.36it/s]


Newness for C07D in 2011; mean: 0.07878416611981363; sd: 0.18805025579562323, mean+sdL: 0.2668344219154369


100%|██████████| 1121/1121 [00:00<00:00, 662414.03it/s]

Uniqueness for C07D in 2011; mean: 0.9613716073796361; sd: 0.1760034007168493, mean+sdL: 1.1373750080964855


...

Difference for B32B in 2016; mean: 0.7057933579335792; sd: 0.3731574879385264, mean+sdL: 1.0789508458721055


100%|██████████| 271/271 [00:00<00:00, 107536.08it/s]


SurpriseNew for B32B in 2016; mean: 0.34525679322727293; sd: 0.33692498271444105, mean+sdL: 0.6821817759417139


100%|██████████| 271/271 [00:00<00:00, 65736.88it/s]

SurpriseDiv for B32B in 2016; mean: 6.0723410376596565; sd: 2.9780125679974456, mean+sdL: 9.050353605657103




In [37]:
import pandas as pd
import glob

# Define the path where your CSV files are stored
path = '/home/edgarlanoue/data/csv/ipcMetrics/train'

# Use glob to find all CSV files in the directory
all_files = glob.glob(f"{path}/*.csv")

# Read each CSV file and store it in a list of DataFrames
df_list = [pd.read_csv(file) for file in all_files]

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Display the first few rows of the combined DataFrame
combined_df

,application_number,label,new_ratio,new_bin,new_len1,uniq_ratio,uniq_bin,diff_ratio,diff_bin,surpNew_ratio,surpNew_bin,surpDiv_ratio,surpDiv_bin
0,14900663,1,0.000000,0,1,1.00,1,0.80,1,0.000000,0,0.000000,0
1,14724494,1,0.000000,0,0,1.00,1,1.00,1,0.476190,1,7.698116,1
2,14896426,1,0.000000,0,0,0.75,1,0.00,0,0.000000,0,1.547181,1
3,14615972,1,0.111111,1,0,1.00,1,1.00,1,0.972222,1,11.318805,1
4,14890354,1,0.600000,1,0,1.00,1,1.00,1,1.000000,1,11.525131,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28917,13263792,0,0.000000,0,1,1.00,1,0.00,0,0.000000,0,0.000000,0
28918,12998018,1,0.000000,0,0,1.00,1,0.97,1,0.000000,0,2.408562,1
28919,13066570,0,0.000000,0,0,1.00,1,1.00,1,0.000000,0,4.647508,1
28920,13336171,0,0.000000,0,1,1.00,1,0.00,0,0.000000,0,0.000000,0


In [50]:
# Step 1: Calculate global statistics for columns ending in 'ratio'
ratio_columns_global = combined_df.filter(regex="ratio$")
global_mean = ratio_columns_global.mean()
global_std = ratio_columns_global.std()
global_mean_plus_sd = global_mean + global_std
global_mean_minus_sd = global_mean - global_std

# Step 2: Create a dictionary for the global stats row
global_stats = {
    "part": "Global",
    **{f"{col}_mean": global_mean[col] for col in global_mean.index},
    **{f"{col}_std": global_std[col] for col in global_std.index},
    **{f"{col}_mean_plus_sd": global_mean_plus_sd[col] for col in global_mean.index},
    **{f"{col}_mean_minus_sd": global_mean_minus_sd[col] for col in global_mean.index}
}

# Step 3: Determine the number of rows in each of the 12 parts
num_parts = 12
rows_per_part = len(combined_df) // num_parts

# Step 4: Initialize a list to store the results and add the global stats as the first entry
summary_stats = [global_stats]

# Step 5: Loop over each part, compute the mean, std, mean+sd, and mean-sd, and store in summary_stats
for i in range(num_parts):
    # Get the subset for the current part
    part_df = combined_df.iloc[i * rows_per_part : (i + 1) * rows_per_part]
    
    # Filter columns ending in 'ratio'
    ratio_columns = part_df.filter(regex="ratio$")
    
    # Calculate mean and standard deviation
    part_mean = ratio_columns.mean()
    part_std = ratio_columns.std()
    
    # Calculate mean + sd and mean - sd
    part_mean_plus_sd = part_mean + part_std
    part_mean_minus_sd = part_mean - part_std
    
    # Append to summary_stats with an identifier for the part
    summary_stats.append({
        "part": i + 1,
        **{f"{col}_mean": part_mean[col] for col in part_mean.index},
        **{f"{col}_std": part_std[col] for col in part_std.index},
        **{f"{col}_mean_plus_sd": part_mean_plus_sd[col] for col in part_mean.index},
        **{f"{col}_mean_minus_sd": part_mean_minus_sd[col] for col in part_mean.index}
    })

# Step 6: Create a DataFrame from the summary_stats list
summary_df = pd.DataFrame(summary_stats)

# Display the summary statistics DataFrame
summary_df[:1]



,part,new_ratio_mean,uniq_ratio_mean,diff_ratio_mean,surpNew_ratio_mean,surpDiv_ratio_mean,new_ratio_std,uniq_ratio_std,diff_ratio_std,surpNew_ratio_std,...,new_ratio_mean_plus_sd,uniq_ratio_mean_plus_sd,diff_ratio_mean_plus_sd,surpNew_ratio_mean_plus_sd,surpDiv_ratio_mean_plus_sd,new_ratio_mean_minus_sd,uniq_ratio_mean_minus_sd,diff_ratio_mean_minus_sd,surpNew_ratio_mean_minus_sd,surpDiv_ratio_mean_minus_sd
0,Global,0.038364,0.960441,0.614012,0.279074,4.542302,0.123379,0.16328,0.411063,0.37071,...,0.161743,1.12372,1.025075,0.649785,8.690208,-0.085015,0.797161,0.202949,-0.091636,0.394396


In [ ]:
thr_new = 0.161743
thr_uniq = 0.85 #arbitrary
thr_diff = 0.85 #arbitrary
thr_surpNew = 0.649785
thr_surpDiv = 8.690208